In [1]:
using BenchmarkTools
using StaticArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [4]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "2768192252143860608", 2, "feature")

## Vecchia funzione

In [5]:
function evalStruct(self::Lar.Struct)
	dim = Lar.checkStruct(self.body)
   	CTM, stack = Matrix{Float64}(LinearAlgebra.I, dim+1, dim+1), []
   	scene = Lar.traversal(CTM, stack, self, [])
	return scene
end

evalStruct (generic function with 1 method)

In [6]:
@btime evalStruct(structure)

  11.799 μs (36 allocations: 2.88 KiB)


2-element Array{Any,1}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [7]:
@code_llvm evalStruct(structure)


;  @ In[5]:1 within `evalStruct'
; Function Attrs: uwtable
define nonnull %jl_value_t* @japi1_evalStruct_2169(%jl_value_t*, %jl_value_t**, i32) #0 {
top:
  %3 = alloca %jl_value_t*, i32 4
  %gcframe = alloca %jl_value_t*, i32 6, align 16
  %4 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %4, i8 0, i32 48, i1 false)
  %5 = alloca %jl_value_t**, align 8
  store volatile %jl_value_t** %1, %jl_value_t*** %5, align 8
  %6 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #5
  %7 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
  %8 = bitcast %jl_value_t** %7 to i64*
  store i64 16, i64* %8
  %9 = getelementptr %jl_value_t**, %jl_value_t*** %6, i32 0
  %10 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
  %11 = bitcast %jl_value_t** %10 to %jl_value_t***
  %12 = load %jl_value_t**, %jl_value_t*** %9
  store %jl_value_t** %12, %jl_value_t*** %11
  %13 = bitcast %jl_value_t*** %9 to %jl_value_t***
  store %j

In [8]:
@benchmark evalStruct(structure)

BenchmarkTools.Trial: 
  memory estimate:  2.88 KiB
  allocs estimate:  36
  --------------
  minimum time:     8.099 μs (0.00% GC)
  median time:      16.100 μs (0.00% GC)
  mean time:        18.822 μs (0.00% GC)
  maximum time:     295.300 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Modifiche

In [9]:
function evalStruct(self)
	dim::Int = Lar.checkStruct(self.body)
	CTM = Matrix{Float64}(LinearAlgebra.I, dim+1, dim+1)
	return Lar.traversal(CTM, [], self, [])
end

evalStruct (generic function with 2 methods)

In [10]:
@btime evalStruct(structure)

  7.850 μs (36 allocations: 2.88 KiB)


2-element Array{Any,1}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [11]:
@code_typed evalStruct(structure)

CodeInfo(
1 ─ %1  = Base.getproperty(Main.Lar, :checkStruct)::Any
│   %2  = Base.getfield(self, :body)::Array
│   %3  = (%1)(%2)::Any
│   %4  = LinearAlgebra.I::Core.Compiler.Const(UniformScaling{Bool}(true), false)
│   %5  = (%3 + 1)::Any
│   %6  = (%3 + 1)::Any
│   %7  = (Array{Float64,2})(%4, %5, %6)::Array{Float64,2}
│   %8  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), 0, :(:ccall), Array{Any,1}, 0, 0))::Array{Any,1}
│   %9  = Base.getproperty(Main.Lar, :traversal)::Any
│   %10 = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), 0, :(:ccall), Array{Any,1}, 0, 0))::Array{Any,1}
│   %11 = (%9)(%7, %8, self, %10)::Any
└──       return %11
) => Any

In [12]:
@code_llvm evalStruct(structure)


;  @ In[5]:1 within `evalStruct'
; Function Attrs: uwtable
define nonnull %jl_value_t* @japi1_evalStruct_2502(%jl_value_t*, %jl_value_t**, i32) #0 {
top:
  %3 = alloca %jl_value_t*, i32 4
  %gcframe = alloca %jl_value_t*, i32 6, align 16
  %4 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %4, i8 0, i32 48, i1 false)
  %5 = alloca %jl_value_t**, align 8
  store volatile %jl_value_t** %1, %jl_value_t*** %5, align 8
  %6 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #5
  %7 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
  %8 = bitcast %jl_value_t** %7 to i64*
  store i64 16, i64* %8
  %9 = getelementptr %jl_value_t**, %jl_value_t*** %6, i32 0
  %10 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
  %11 = bitcast %jl_value_t** %10 to %jl_value_t***
  %12 = load %jl_value_t**, %jl_value_t*** %9
  store %jl_value_t** %12, %jl_value_t*** %11
  %13 = bitcast %jl_value_t*** %9 to %jl_value_t***
  store %j

In [13]:
@benchmark evalStruct(structure)

BenchmarkTools.Trial: 
  memory estimate:  2.88 KiB
  allocs estimate:  36
  --------------
  minimum time:     7.625 μs (0.00% GC)
  median time:      9.075 μs (0.00% GC)
  mean time:        11.159 μs (3.05% GC)
  maximum time:     1.853 ms (97.62% GC)
  --------------
  samples:          10000
  evals/sample:     4